In [1]:
import pandas as pd
import requests
import os
import numpy as np


pd.set_option('display.max_columns', None)

In [2]:
caminho = "arquivos/preproc"
pesquisas = "arquivos/dados3.xlsx"
sheets = pd.ExcelFile(pesquisas).sheet_names
arquivos = []
for item in os.listdir(caminho):
    arquivos.append(item)

In [3]:
# def anoEleicao(ano, anos_validos):
#     try:
#         anos_ordenados = sorted(anos_validos)
#         menores = [a for a in anos_ordenados if a <= int(ano)]
#         return max(menores) if menores else None
#     except:
#         return 'Atual'

def formatar_data_ptbr(data):
    meses = {
        'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04',
        'mai': '05', 'jun': '06', 'jul': '07', 'ago': '08',
        'set': '09', 'out': '10', 'nov': '11', 'dez': '12'
    }

    if pd.isna(data):
        return None

    try:
        dia, mes, ano = data.lower().split()
        return f"{dia.zfill(2)}/{meses[mes]}/{ano}"
    except Exception:
        return None

eleicoes = [
    # [Ano, Tipo (1=Municipal, 2=Gerais), Data 1º Turno, Data 2º Turno (se houver)]
    [2008, 1, "05/10/2008", "26/10/2008"],  # municipais
    [2010, 2, "03/10/2010", "31/10/2010"],  # gerais
    [2012, 1, "07/10/2012", "28/10/2012"],  # municipais
    [2014, 2, "05/10/2014", "26/10/2014"],  # gerais
    [2016, 1, "02/10/2016", "30/10/2016"],  # municipais
    [2018, 2, "07/10/2018", "28/10/2018"],  # gerais
    [2020, 1, "15/11/2020", "29/11/2020"],  # municipais (exceção de adiamento)
    [2022, 2, "02/10/2022", "30/10/2022"],  # gerais
    [2024, 1, "06/10/2024", "27/10/2024"],  # municipais
    [2026, 2, "04/10/2026", "25/10/2026"]   # gerais (previstas)
]
eleicoes = pd.DataFrame(eleicoes)
eleicoes = eleicoes.rename(columns={
    0: 'aa_eleicao',
    1: 'tipo',
    2: 'data1',
    3: 'data2',
})

estados = [
	['AC', 'RIO BRANCO'],
	['AL', 'MACEIÓ'],
	['AP', 'MACAPÁ'],
	['AM', 'MANAUS'],
	['BA', 'SALVADOR'],
	['CE', 'FORTALEZA'],
	['DF', 'BRASÍLIA'],
	['ES', 'VITÓRIA'],
	['GO', 'GOIÂNIA'],
	['MA', 'SÃO LUÍS'],
	['MT', 'CUIABÁ'],
	['MS', 'CAMPO GRANDE'],
	['MG', 'BELO HORIZONTE'],
	['PA', 'BELÉM'],
	['PB', 'JOÃO PESSOA'],
	['PR', 'CURITIBA'],
	['PE', 'RECIFE'],
	['PI', 'TERESINA'],
	['RJ', 'RIO DE JANEIRO'],
	['RN', 'NATAL'],
	['RS', 'PORTO ALEGRE'],
	['RO', 'PORTO VELHO'],
	['RR', 'BOA VISTA'],
	['SC', 'FLORIANÓPOLIS'],
	['SP', 'SÃO PAULO'],
	['SE', 'ARACAJU'],
	['TO', 'PALMAS'],
]

In [4]:
resumo = pd.DataFrame()
for arq in arquivos:
    df = pd.read_csv(caminho+'/'+arq, encoding='latin-1',sep=';')
    if "prefeito" in arq:
        lista_res = []
        for estado in estados:
            tmp = df[(df["nm_municipio"] == estado[1]) & (df["sg_uf"] == estado[0])]
            lista_res.append(tmp)
        res = pd.concat(lista_res, ignore_index=True)
    elif "presidente" in arq:    
        df["sg_uf"] = "BR"
        df["nm_municipio"] = "Brasil"
        res = df[(df["cd_cargo"] == 1)]
    elif "governador" in arq:
        df["nm_municipio"] = "Capital do estado"
        res = df[(df["cd_cargo"] == 3)]
    resumo = pd.concat([resumo, res], ignore_index=True)
resumo["pc_votos_validos"] = (resumo["pc_votos_validos"].str.replace(",", ".", regex=False).astype(float))
resumo["pc_votos_validos100"] = resumo["pc_votos_validos"] * 100
resumo["pc_votos_validos100"] = resumo["pc_votos_validos100"].round(2)

In [5]:
baseDeDados = pd.DataFrame()
lista_dfs = []
for sheet in sheets[1:]:
    print(sheet)
    pesquisa = pd.read_excel(pesquisas, sheet_name=sheet)
    pesquisa[10] = pesquisa[10].str.upper()
    pesquisa = pesquisa[pesquisa[10] != 'VOTO BRANCO/NULO']

    pesquisa[3] = pesquisa[3].str.replace("º", "", regex=False).astype(int)
    pesquisa[11] = pesquisa[11].str.replace("%", "", regex=False)
    pesquisa[11] = pesquisa[11].str.replace("0 ou Nulo", '0', regex=False)
    pesquisa[11] = (pesquisa[11].str.replace(",", ".", regex=False).astype(float))
    pesquisa[7] = (pesquisa[7].str.replace(",", ".", regex=False).astype(str))
    pesquisa[7] = pesquisa[7].str.replace(r'^\s*null\s+pp\s*$', '0', regex=True)
    pesquisa[8] = (pesquisa[8].str.replace(",", ".", regex=False).astype(str))
    try:
        pesquisa[7] = (
            pesquisa[7]
            .astype(str)
            .str.replace(",", ".", regex=False)
            .str.replace(r'(?i)\bpp\b', '', regex=True)   # remove pp / Pp / PP
            .str.replace(r'^\s*null\s*$', '', regex=True) # remove Null
            .str.strip()
        )
        pesquisa[7] = pd.to_numeric(pesquisa[7], errors='coerce')
    except:
         pesquisa[7] = pesquisa[7]
    pesquisa[8] = (pesquisa[8].str.replace("p.p.", "", regex=False).astype(float))
    pesquisa["aa_eleicao"] = pesquisa[4].astype(str).str.extract(r'(\d{4})').astype(int)
    pesquisa["pesquisa"] = sheet
    pesquisa = pesquisa.rename(columns={
        0: "idPesquisa",
        1: "ds_cargo",
        2: "sg_uf",
        3: "nr_turno",
        4: "dataPesquisa",
        5: "amostra",
        6: "metodologia",
        7: "Margem de erro",
        8: "Erro pesquisa",
        10: "nm_urna_candidato",
        11: "percentualPesquisa"
    })
    # pesquisa["ano_eleito"] = pesquisa['aa_eleicao'].apply(
    #     lambda x: anoEleicao((x), eleicoes)
    # )
    df = pd.merge(pesquisa, resumo, on=["ds_cargo", "sg_uf", "nr_turno", "nm_urna_candidato","aa_eleicao"], how='left')
    #df.to_excel('arquivos/enriquecidos/'+sheet+'.xlsx')
    lista_dfs.append(df)
baseDeDados = pd.concat(lista_dfs, ignore_index=True)

Datafolha
AtlasIntel
MDA
Paraná Pesquisas
REAL TIME
Futura
6 Sigma
Instituto Opinião (PR)
Ipespe
Quaest
Veritá
Estimativa
Ranking Pesquisa
Datavox (PB)
Voga Pesquisas
Brasmarket
DataPoder360
FSB Pesquisa
GERP
Ideia Big Data
Ipec (antigo Ibope)
Sensus


In [6]:
baseDeDados = pd.concat(lista_dfs, ignore_index=True)
# Ordenação e ranking
baseDeDados = baseDeDados.sort_values(
    by=['idPesquisa', 'pesquisa', 'percentualPesquisa'],
    ascending=[True, True, False]
)

baseDeDados['ranking'] = (
    baseDeDados
    .groupby(['idPesquisa', 'pesquisa'])
    .cumcount() + 1
)

# Acerto da pesquisa
baseDeDados['Acertou'] = False
baseDeDados.loc[
    (baseDeDados['ranking'] == 1) &
    (baseDeDados['ds_sit_totalizacao'] == 'Eleito'),
    'Acertou'
] = True

baseDeDados.loc[
    ((baseDeDados['ranking'] == 1) | (baseDeDados['ranking'] == 2)) &
    (baseDeDados['ds_sit_totalizacao'] == 'Segundo turno'),
    'Acertou'
] = True

# ===== Datas =====

baseDeDados = pd.merge(baseDeDados, eleicoes, on=["aa_eleicao"], how='left')

# Data da pesquisa (pt-BR -> datetime)
baseDeDados['dataPesquisa'] = (
    baseDeDados['dataPesquisa']
    .apply(formatar_data_ptbr)
    .pipe(pd.to_datetime, dayfirst=True, errors='coerce')
)

# Datas da eleição
baseDeDados['data1'] = pd.to_datetime(
    baseDeDados['data1'], dayfirst=True, errors='coerce'
)
baseDeDados['data2'] = pd.to_datetime(
    baseDeDados['data2'], dayfirst=True, errors='coerce'
)

# Seleciona a data correta conforme o turno
baseDeDados['data_eleicao'] = np.where(
    baseDeDados['nr_turno'] == 1,
    baseDeDados['data1'],
    baseDeDados['data2']
)

# Distância em dias entre pesquisa e eleição
baseDeDados['DistanciaDataPesquisaEleicao'] = (
    baseDeDados['data_eleicao'] - baseDeDados['dataPesquisa']
).dt.days

baseDeDados["pc_votos_validos100"] = baseDeDados["pc_votos_validos100"].fillna(0)
baseDeDados["diferencaPesquisaEleicao"] = baseDeDados['percentualPesquisa']-baseDeDados["pc_votos_validos100"]

In [7]:
baseDeDados

,idPesquisa,ds_cargo,sg_uf,nr_turno,dataPesquisa,amostra,metodologia,Margem de erro,Erro pesquisa,9,nm_urna_candidato,percentualPesquisa,aa_eleicao,pesquisa,cd_cargo,nr_candidato,nm_candidato,sg_partido,ds_composicao_coligacao,ds_sit_totalizacao,dt_ult_totalizacao,sg_ue,sq_candidato,nm_tipo_destinacao_votos,sq_eleicao_divulga,pc_votos_validos,qt_votos_nom_validos,qt_votos_concorrentes,nm_municipio,pc_votos_validos100,ranking,Acertou,tipo,data1,data2,data_eleicao,DistanciaDataPesquisaEleicao,diferencaPesquisaEleicao
0,0,Presidente,BR,1,2022-08-06,1.0,Fone,3.1,12.2,-,JAIR BOLSONARO,37.5,2022,6 Sigma,1.0,22.0,JAIR MESSIAS BOLSONARO,PL,PP / REPUBLICANOS / PL,Segundo turno,2022-10-04 12:06:36,BR,2.800016e+11,Válido,2.040602e+09,0.4320,51072345.0,118229719.0,Brasil,43.20,1,True,2,2022-10-02,2022-10-30,2022-10-02,57,-5.70
1,0,Presidente,BR,1,2022-08-06,1.0,Fone,3.1,12.2,-,LULA,28.2,2022,6 Sigma,1.0,13.0,LUIZ INÁCIO LULA DA SILVA,PT,Federação Brasil da Esperança - FE BRASIL(PT/P...,Segundo turno,2022-10-04 12:06:36,BR,2.800016e+11,Válido,2.040602e+09,0.4843,57259504.0,118229719.0,Brasil,48.43,2,True,2,2022-10-02,2022-10-30,2022-10-02,57,-20.23
2,0,Presidente,BR,1,2022-08-06,1.0,Fone,3.1,12.2,-,CIRO GOMES,8.0,2022,6 Sigma,1.0,12.0,CIRO FERREIRA GOMES,PDT,PDT,Não Eleito,2022-10-04 12:06:36,BR,2.800016e+11,Válido,2.040602e+09,0.0304,3599287.0,118229719.0,Brasil,3.04,3,False,2,2022-10-02,2022-10-30,2022-10-02,57,4.96
3,0,Presidente,BR,1,2022-08-06,1.0,Fone,3.1,12.2,-,SIMONE TEBET,3.0,2022,6 Sigma,1.0,15.0,SIMONE NASSAR TEBET,MDB,MDB / Federação PSDB Cidadania(PSDB/CIDADANIA)...,Não Eleito,2022-10-04 12:06:36,BR,2.800016e+11,Válido,2.040602e+09,0.0416,4915423.0,118229719.0,Brasil,4.16,4,False,2,2022-10-02,2022-10-30,2022-10-02,57,-1.16
4,0,Presidente,BR,1,2022-08-06,1.0,Fone,3.1,12.2,-,PABLO MARCAL,1.1,2022,6 Sigma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,5,False,2,2022-10-02,2022-10-30,2022-10-02,57,1.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19183,427,Governador,BA,1,2018-08-18,1.2,Fone,3.0,3.5,-,JOÃO HENRIQUE DURVAL CARNEIRO,1.0,2018,REAL TIME,3.0,28.0,JOÃO HENRIQUE DE BARRADAS CARNEIRO,PRTB,PRTB / PSL,Não Eleito,2020-11-06 14:05:14,BA,5.000062e+10,Válido,2.022802e+09,0.0058,38960.0,6749554.0,Capital do estado,0.58,3,False,2,2018-10-07,2018-10-28,2018-10-07,50,0.42
19184,427,Governador,BA,1,2018-08-18,1.2,Fone,3.0,3.5,-,MARCOS MENDES,0.0,2018,REAL TIME,3.0,50.0,MARCOS ANTONIO GUIMARAES MENDES,PSOL,PSOL,Não Eleito,2020-11-06 14:05:14,BA,5.000060e+10,Válido,2.022802e+09,0.0066,44702.0,6737327.0,Capital do estado,0.66,4,False,2,2018-10-07,2018-10-28,2018-10-07,50,-0.66
19185,427,Governador,BA,1,2018-08-18,1.2,Fone,3.0,3.5,-,ORLANDO ANDRADE,0.0,2018,REAL TIME,3.0,29.0,ORLANDO ANDRADE DE JESUS,PCO,PCO,Não Eleito,2020-11-06 14:05:14,BA,5.000062e+10,Válido,2.022802e+09,0.0005,3100.0,6150664.0,Capital do estado,0.05,5,False,2,2018-10-07,2018-10-28,2018-10-07,50,-0.05
19186,427,Governador,BA,1,2018-08-18,1.2,Fone,3.0,3.5,-,CÉLIA SACRAMENTO,0.0,2018,REAL TIME,3.0,18.0,CÉLIA OLIVEIRA DE JESUS SACRAMENTO,REDE,REDE,Não Eleito,2020-11-06 14:05:14,BA,5.000062e+10,Válido,2.022802e+09,0.0046,31198.0,6749554.0,Capital do estado,0.46,6,False,2,2018-10-07,2018-10-28,2018-10-07,50,-0.46


In [16]:
baseDeDados.to_excel('arquivos/enriquecidos/baseDeDados.xlsx')